In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

dates_df = pd.DataFrame(columns = ['Measure ID', 'Start Date', 'End Date'])

In [2]:
def curate(df):

    try:
        df = df[df['PROVIDER_ID'] != np.nan]
        df['PROVIDER_ID'] = df['PROVIDER_ID'].values.astype(str)
        
        ids = df['PROVIDER_ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['PROVIDER_ID'] = ids2
        
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    if 'Unnamed: 0' in list(df):
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    return df

# Use 2025 SAS input file to get the column labels needed for 2026 prognostication

Remove OP-2 and OP-3B, as these will no longer be reported in Care Compare.   
Will also need to add measures not included in 2025.

In [3]:
sas_input_df = pd.read_sas(stars_dir + '2025/2025-07 Stars Release/alldata_2025jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")


ls = ['OP_2', 'OP_3B', 'OP_2_DEN', 'OP_3B_DEN', 
      'PC_01', 'PC_01_DEN',
      'READM_30_HOSP_WIDE', 'READM_30_HOSP_WIDE_DEN',
      'H_NUMB_COMP', 'H_RESP_RATE_P',
     ]
for l in ls:
    try:
        sas_input_df.drop(labels=[l], axis=1, inplace=True)
    except:
        pass
    
sas_cols_2026 = list(sas_input_df)

ls = ['Hybrid_HWR',
      'ePC_02', #'PC_05', 
      #'ePC_07a', #'ePC_07b',
      #'HH-01', 'HH-02',
      #'Hybrid_HWM',
     ]

for l in ls:
    sas_cols_2026.append(l)


print(len(sas_cols_2026), 'columns in 2025 Stars SAS input file (not all get used by the SAS programs):')
print(sas_cols_2026, '\n')

print('Create columns for 2025 data')

sas_cols = list(sas_cols_2026)

sas_input_df = curate(sas_input_df)
prvdrs_2025 = sas_input_df['PROVIDER_ID'].unique().tolist()

print(len(prvdrs_2025), 'hospitals in 2025 Stars output file\n')
prvdrs_2025 = sorted(list(set(prvdrs_2025)))

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')

sas_input_df.head()

89 columns in 2025 Stars SAS input file (not all get used by the SAS programs):
['PROVIDER_ID', 'COMP_HIP_KNEE', 'IMM_3_DEN', 'EDAC_30_AMI', 'EDAC_30_AMI_DEN', 'HAI_3', 'EDAC_30_HF_DEN', 'OP_18B_DEN', 'HAI_6', 'OP_29', 'OP_23_DEN', 'SAFE_USE_OF_OPIOIDS', 'MORT_30_COPD_DEN', 'HAI_3_DEN_VOL', 'HAI_5_DEN_PRED', 'OP_35_ED', 'OP_13_DEN', 'READM_30_CABG', 'HAI_5_DEN_VOL', 'OP_22', 'HAI_1_DEN_VOL', 'OP_36', 'MORT_30_STK', 'READM_30_COPD', 'READM_30_HIP_KNEE_DEN', 'HAI_6_DEN_VOL', 'MORT_30_PN', 'OP_35_ADM', 'MORT_30_HF_DEN', 'OP_32', 'SAFE_USE_OF_OPIOIDS_DEN', 'H_COMP_7_STAR_RATING', 'READM_30_HIP_KNEE', 'HCP_COVID_19', 'READM_30_COPD_DEN', 'H_COMP_5_STAR_RATING', 'OP_10_DEN', 'COMP_HIP_KNEE_DEN', 'HAI_2_DEN_PRED', 'MORT_30_CABG', 'OP_29_DEN', 'OP_22_DEN', 'SEP_1', 'OP_23', 'MORT_30_AMI_DEN', 'H_COMP_6_STAR_RATING', 'OP_18B', 'MORT_30_AMI', 'MORT_30_PN_DEN', 'EDAC_30_PN', 'OP_36_DEN', 'MORT_30_CABG_DEN', 'HAI_1_DEN_PRED', 'HAI_6_DEN_PRED', 'MORT_30_COPD', 'OP_13', 'PSI_90_SAFETY', 'MORT_30_HF'

,PROVIDER_ID,COMP_HIP_KNEE,IMM_3_DEN,EDAC_30_AMI,EDAC_30_AMI_DEN,HAI_3,EDAC_30_HF_DEN,OP_18B_DEN,HAI_6,OP_29,OP_23_DEN,SAFE_USE_OF_OPIOIDS,MORT_30_COPD_DEN,HAI_3_DEN_VOL,HAI_5_DEN_PRED,OP_35_ED,OP_13_DEN,READM_30_CABG,HAI_5_DEN_VOL,OP_22,HAI_1_DEN_VOL,OP_36,MORT_30_STK,READM_30_COPD,READM_30_HIP_KNEE_DEN,HAI_6_DEN_VOL,MORT_30_PN,OP_35_ADM,MORT_30_HF_DEN,OP_32,SAFE_USE_OF_OPIOIDS_DEN,H_COMP_7_STAR_RATING,READM_30_HIP_KNEE,HCP_COVID_19,READM_30_COPD_DEN,H_COMP_5_STAR_RATING,OP_10_DEN,COMP_HIP_KNEE_DEN,HAI_2_DEN_PRED,MORT_30_CABG,OP_29_DEN,OP_22_DEN,SEP_1,OP_23,MORT_30_AMI_DEN,H_COMP_6_STAR_RATING,OP_18B,MORT_30_AMI,MORT_30_PN_DEN,EDAC_30_PN,OP_36_DEN,MORT_30_CABG_DEN,HAI_1_DEN_PRED,HAI_6_DEN_PRED,MORT_30_COPD,OP_13,PSI_90_SAFETY,MORT_30_HF,HCP_COVID_19_DEN,H_COMP_1_STAR_RATING,READM_30_CABG_DEN,PSI_90_SAFETY_DEN,IMM_3,HAI_3_DEN_PRED,H_COMP_3_STAR_RATING,OP_8_DEN,MORT_30_STK_DEN,OP_8,PSI_4_SURG_COMP_DEN,OP_10,EDAC_30_HF,HAI_2_DEN_VOL,OP_32_DEN,OP_35_ED_DEN,HAI_2,H_COMP_2_STAR_RATING,HAI_1,PSI_4_SURG_COMP,OP_35_ADM_DEN,SEP_1_DEN,EDAC_30_PN_DEN,HAI_5,HAI_4,HAI_4_DEN_VOL,HAI_4_DEN_PRED,H_INDI_STAR_RATING,H_GLOB_STAR_RATING
0,010001,0.030,4115.0,-13.8,296.0,1.209,679.0,345.0,0.491,0.47,NaN,0.12,122.0,240.0,11.232,4.9,193.0,0.102,103195.0,0.05,9538.0,1.1,0.137,0.177,34.0,103195.0,0.189,11.9,610.0,12.9,4303.0,4.0,0.044,0.000,130.0,3.0,1478.0,32.0,23.712,0.038,17.0,52960.0,0.68,NaN,291.0,3.0,217.0,0.108,489.0,14.3,668.0,157.0,10.082,65.234,0.079,0.021,0.98,0.105,2712.0,2.0,151.0,NaN,0.96,6.618,3.0,66.0,414.0,0.333,125.0,0.054,10.6,16332.0,170.0,202.0,0.169,4.0,0.496,194.78,202.0,131.0,490.0,0.445,NaN,NaN,NaN,3.0,4.0
1,010005,0.027,2407.0,NaN,NaN,0.000,176.0,1154.0,0.631,0.96,13.0,0.16,133.0,102.0,1.631,5.5,177.0,NaN,35424.0,0.03,3410.0,1.9,0.139,0.161,144.0,33872.0,0.231,7.9,162.0,14.2,2244.0,3.0,0.039,0.026,143.0,4.0,850.0,138.0,3.262,NaN,180.0,56820.0,0.76,0.69,28.0,4.0,144.0,0.145,301.0,-8.4,406.0,NaN,2.074,9.503,0.092,0.028,1.01,0.127,2431.0,3.0,NaN,NaN,0.72,2.867,2.0,109.0,97.0,0.459,43.0,0.134,9.8,7200.0,739.0,107.0,1.226,4.0,0.964,191.14,107.0,288.0,305.0,2.452,NaN,NaN,NaN,3.5,3.0
2,010006,0.040,2560.0,13.4,315.0,0.000,508.0,349.0,0.026,0.85,NaN,0.16,141.0,73.0,5.397,NaN,243.0,0.115,68020.0,0.01,4826.0,1.4,0.142,0.177,90.0,64728.0,0.198,NaN,455.0,12.1,3521.0,2.0,0.044,0.000,154.0,2.0,973.0,84.0,11.164,0.052,82.0,42286.0,0.56,NaN,292.0,2.0,177.0,0.166,616.0,4.5,484.0,102.0,5.279,37.771,0.102,0.029,1.16,0.122,2095.0,2.0,91.0,NaN,0.66,1.935,2.0,NaN,271.0,NaN,102.0,0.108,-4.9,8700.0,1355.0,NaN,0.000,2.0,0.000,201.13,NaN,162.0,621.0,0.371,0.0,116.0,1.086,2.5,2.0
3,010007,0.034,345.0,NaN,NaN,NaN,35.0,594.0,1.003,0.23,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,0.04,NaN,1.2,NaN,0.190,33.0,5267.0,0.290,NaN,33.0,13.4,NaN,4.0,0.044,0.000,41.0,5.0,151.0,33.0,NaN,NaN,111.0,11202.0,0.24,NaN,NaN,5.0,129.0,NaN,101.0,21.5,59.0,NaN,NaN,1.994,0.135,NaN,0.93,0.141,245.0,5.0,NaN,NaN,0.26,NaN,4.0,NaN,NaN,NaN,NaN,0.073,20.9,NaN,109.0,NaN,NaN,5.0,NaN,NaN,NaN,21.0,85.0,NaN,NaN,NaN,NaN,3.0,4.0
4,010008,NaN,126.0,NaN,NaN,NaN,NaN,329.0,NaN,0.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.192,NaN,NaN,12.9,NaN,NaN,NaN,0.019,NaN,NaN,74.0,NaN,NaN,NaN,24.0,6239.0,NaN,NaN,NaN,NaN,121.0,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,NaN,0.39,NaN,NaN,NaN,NaN,NaN,NaN,0.014,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## HAIs

In [4]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Healthcare_Associated_Infections-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['HAI_1_ELIGCASES', 'HAI_1_DOPC', 'HAI_1_SIR', 'HAI_2_ELIGCASES', 'HAI_2_DOPC', 'HAI_2_SIR', 
            'HAI_3_ELIGCASES', 'HAI_3_DOPC', 'HAI_3_SIR', 'HAI_4_ELIGCASES', 'HAI_4_DOPC', 'HAI_4_SIR', 
            'HAI_5_ELIGCASES', 'HAI_5_DOPC', 'HAI_5_SIR', 'HAI_6_ELIGCASES', 'HAI_6_DOPC', 'HAI_6_SIR']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

hai_df = pd.DataFrame(columns=['Facility ID']) 
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    hai_df = hai_df.merge(tdf2, on='Facility ID', how='outer')
    
hai_df.rename(columns={'HAI_1_ELIGCASES': 'HAI_1_DEN_PRED',
                       'HAI_1_DOPC': 'HAI_1_DEN_VOL',
                       'HAI_1_SIR': 'HAI_1',
                       'HAI_2_ELIGCASES': 'HAI_2_DEN_PRED',
                       'HAI_2_DOPC': 'HAI_2_DEN_VOL',
                       'HAI_2_SIR': 'HAI_2',
                       'HAI_3_ELIGCASES': 'HAI_3_DEN_PRED',
                       'HAI_3_DOPC': 'HAI_3_DEN_VOL',
                       'HAI_3_SIR': 'HAI_3',
                       'HAI_4_ELIGCASES': 'HAI_4_DEN_PRED',
                       'HAI_4_DOPC': 'HAI_4_DEN_VOL',
                       'HAI_4_SIR': 'HAI_4',
                       'HAI_5_ELIGCASES': 'HAI_5_DEN_PRED',
                       'HAI_5_DOPC': 'HAI_5_DEN_VOL',
                       'HAI_5_SIR': 'HAI_5',
                       'HAI_6_ELIGCASES': 'HAI_6_DEN_PRED',
                       'HAI_6_DOPC': 'HAI_6_DEN_VOL',
                       'HAI_6_SIR': 'HAI_6',
                       'Facility ID': 'PROVIDER_ID',
                   }, inplace=True)

for c in list(hai_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
hai_df = curate(hai_df)

70 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'EDAC_30_AMI', 'EDAC_30_AMI_DEN', 'EDAC_30_HF', 'EDAC_30_HF_DEN', 'EDAC_30_PN', 'EDAC_30_PN_DEN', 'HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'Hybrid_HWR', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_32', 'OP_32_DEN', 'OP_35_ADM', 'OP_35_ADM_DEN', 'OP_35_ED', 'OP_35_ED_DEN', 'OP_36', 'OP_36_DEN', 'OP_8', 'OP_8_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 'READM_30_CABG', 'READM_30_CABG_DEN', 'R

## Unplanned Hospital Visits


In [5]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Unplanned_Hospital_Visits-Hospital.csv')
measures = ['EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 
            'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'Hybrid_HWR']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Denominator', 'Measure ID', 'Score'], axis=1)

uhv_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    uhv_df = uhv_df.merge(tdf2, on='Facility ID', how='outer')

uhv_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(uhv_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass
    
print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
uhv_df = curate(uhv_df)

49 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_8', 'OP_8_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 'SAFE_USE_OF_OPIOIDS', 'SAFE_USE_OF_OPIOIDS_DEN', 'SEP_1', 'SEP_1_DEN', 'ePC_02'] 



## COMPLICATIONS AND DEATHS

In [6]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Complications_and_Deaths-Hospital.csv')

measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 
            'MORT_30_PN', 'MORT_30_STK', 'PSI_04', 'COMP_HIP_KNEE',
            'PSI_90', 
            #'Hybrid_HWM',
           ]

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Denominator'], axis=1)

cad_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    cad_df = cad_df.merge(tdf2, on='Facility ID', how='outer')
    
cad_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'PSI_04': 'PSI_4_SURG_COMP',
                       'PSI_04_DEN': 'PSI_4_SURG_COMP_DEN',
                       'PSI_90': 'PSI_90_SAFETY',
                       'PSI_90_DEN': 'PSI_90_SAFETY_DEN',
                   }, inplace=True)

for c in list(cad_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass
    
print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
cad_df = curate(cad_df)

31 remaining features: ['HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'IMM_3', 'IMM_3_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_8', 'OP_8_DEN', 'SAFE_USE_OF_OPIOIDS', 'SAFE_USE_OF_OPIOIDS_DEN', 'SEP_1', 'SEP_1_DEN', 'ePC_02'] 



## TIMELY AND EFFECTIVE CARE

Everything except PC-01, which for 2024 is located in the Maternal Health files of the hospitals data archive

In [7]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Timely_and_Effective_Care-Hospital.csv')

df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Timely_and_Effective_Care-Hospital.csv')

measures = ['IMM_3', 'OP_18b', 'OP_22', 'OP_23', 'OP_29', 'SEP_1', 'HCP_COVID_19',
            'SAFE_USE_OF_OPIOIDS', 
            #'HH-01', 'HH-02',
           ]

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)

ls = tdf['Measure ID'].unique().tolist()
print(ls)

['HCP_COVID_19', 'IMM_3', 'OP_18b', 'OP_22', 'OP_23', 'OP_29', 'SAFE_USE_OF_OPIOIDS', 'SEP_1']


In [8]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Timely_and_Effective_Care-Hospital.csv')

measures = ['IMM_3', 'OP_18b', 'OP_22', 'OP_23', 'OP_29', 'SEP_1', 'HCP_COVID_19',
            'SAFE_USE_OF_OPIOIDS', 
            #'HH-01', 'HH-02',
           ]

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Sample', 'Measure ID', 'Score'], axis=1)

tec_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    if m == 'HCP_COVID_19':
        pass
    else:
        tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec_df = tec_df.merge(tdf2, on='Facility ID', how='outer')
    
tec_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                       'OP_18b': 'OP_18B',
                       'OP_18b_DEN': 'OP_18B_DEN',
                      }, inplace=True)

for c in list(tec_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
tec_df = curate(tec_df)


16 remaining features: ['HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN', 'ePC_02'] 



In [9]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/VA_TE.csv')

measures = ['IMM-3', 'OP-18b', 'OP-22', 'OP-23', 'OP-29', 'SEP-1', 'HCP-COVID-19']
# Does not have 'SAFE_USE_OF_OPIOIDS', 'HH-01', 'HH-02'

tec3_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec3_df = tec3_df.merge(tdf2, on='Facility ID', how='outer')
    
tec3_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                        'OP-18b': 'OP_18B',
                        'OP-18b_DEN': 'OP_18B_DEN',
                        
                        'IMM-3': 'IMM_3', 
                        'IMM-3_DEN': 'IMM_3_DEN', 
                        
                        'OP-22': 'OP_22', 
                        'OP-22_DEN': 'OP_22_DEN', 
                        
                        'OP-23': 'OP_23', 
                        'OP-23_DEN': 'OP_23_DEN', 
                        
                        'OP-29': 'OP_29', 
                        'OP-29_DEN': 'OP_29_DEN', 
                        
                        'SEP-1': 'SEP_1', 
                        'SEP-1_DEN': 'SEP_1_DEN', 
                        
                        'HCP-COVID-19': 'HCP_COVID_19',
                        'HCP-COVID-19_DEN': 'HCP_COVID_19_DEN',
                        
                      }, inplace=True)

tec3_df.replace('Not Available', np.nan, inplace=True)
print(tec3_df.shape)
tec3_df = curate(tec3_df)
print(tec3_df.shape)

print(tec_df.shape)
print(tec3_df.shape)
tec_df = pd.concat([tec_df, tec3_df])
print(tec_df.shape)

tec_df.head()

(132, 15)
(132, 15)
(4623, 16)
(132, 15)
(4755, 17)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,HCP_COVID_19,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,HCP_COVID_19_DEN
0,010001,96,4115,215,373,5,57084,Not Available,Not Available,72,29,67,147,0.3,12,4303,NaN
1,010005,72,2407,139,1125,3,58624,43,14,100,210,74,278,0.2,16,2244,NaN
2,010006,66,2560,156,342,1,44924,62,16,86,85,62,140,0.5,16,3521,NaN
3,010007,26,345,113,854,2,12667,Not Available,Not Available,65,52,11,18,0,Not Available,Not Available,NaN
4,010008,39,126,114,343,1,6062,Not Available,Not Available,67,12,Not Available,Not Available,0.8,Not Available,Not Available,NaN


## TIMELY AND EFFECTIVE CARE

Only ePC-02, which is located in the Maternal Health files of the hospitals data archive   
Maybe ePC-07a

In [16]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Maternal_Health-Hospital.csv')
print(df['Measure ID'].unique().tolist())

df = df[df['Measure ID'] == 'ePC_02']

ls = sorted(df['Score'].unique().tolist())

print(len(ls))
print(ls)

df.head()

['PC_05', 'SM_7', 'ePC_02', 'ePC_07a', 'ePC_07b']
33
['0', '11', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '35', '36', '38', '45', '5', '6', '7', '73', '8', 'Not Available']


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Measure ID,Measure Name,Score,Sample,Footnote,Start Date,End Date
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,ePC_02,Cesarean Birth,Not Available,Not Available,5.0,01/01/2023,12/31/2023
7,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,ePC_02,Cesarean Birth,Not Available,Not Available,5.0,01/01/2023,12/31/2023
12,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,ePC_02,Cesarean Birth,Not Available,Not Available,5.0,01/01/2023,12/31/2023
17,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,ePC_02,Cesarean Birth,Not Available,Not Available,5.0,01/01/2023,12/31/2023
22,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,ePC_02,Cesarean Birth,Not Available,Not Available,5.0,01/01/2023,12/31/2023


In [10]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Maternal_Health-Hospital.csv')
measures = ['ePC_02', 
            #'ePC_07a', 
            #'ePC_07b', 
            #'PC_05',
           ]

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Sample'], axis=1)

tec2_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    tec2_df = tec2_df.merge(tdf2, on='Facility ID', how='outer')
    
tec2_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(tec2_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')

tec2_df = curate(tec2_df)


15 remaining features: ['HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## HCAHPS

In [11]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/HCAHPS-Hospital.csv')

measures = ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 
            'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 
            'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING']

tdf = df[df['HCAHPS Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf.rename(columns={'HCAHPS Measure ID': 'Measure ID'}, inplace=True)
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['HCAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'HCAHPS Measure ID', 'Patient Survey Star Rating', 
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

HCAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['HCAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Patient Survey Star Rating'].tolist()
    #if i == 0:
    #    tdf2['H_NUMB_COMP'] = tdf1['Number of Completed Surveys'].tolist()
    #    tdf2['H_RESP_RATE_P'] = tdf1['Survey Response Rate Percent'].tolist()
        
    HCAHPS_df = HCAHPS_df.merge(tdf2, on='Facility ID', how='outer')
    

HCAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'] = HCAHPS_df['H_HSP_RATING_STAR_RATING'].astype(int)
HCAHPS_df['H_RECMND_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_RECMND_STAR_RATING'] = HCAHPS_df['H_RECMND_STAR_RATING'].astype(int)
HCAHPS_df['H_GLOB_STAR_RATING'] = np.round((HCAHPS_df['H_HSP_RATING_STAR_RATING'] + HCAHPS_df['H_RECMND_STAR_RATING']) / 2, 1)
HCAHPS_df['H_GLOB_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df['H_CLEAN_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_CLEAN_STAR_RATING'] = HCAHPS_df['H_CLEAN_STAR_RATING'].astype(int)
HCAHPS_df['H_QUIET_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_QUIET_STAR_RATING'] = HCAHPS_df['H_QUIET_STAR_RATING'].astype(int)
HCAHPS_df['H_INDI_STAR_RATING'] = np.round((HCAHPS_df['H_CLEAN_STAR_RATING'] + HCAHPS_df['H_QUIET_STAR_RATING']) / 2, 1)
HCAHPS_df['H_INDI_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df.drop(labels = ['H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING'], axis=1, inplace=True)

for c in list(HCAHPS_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
HCAHPS_df = curate(HCAHPS_df)

7 remaining features: ['HCP_COVID_19_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## Outpatient Imaging Efficiency


In [12]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_08_2025/Outpatient_Imaging_Efficiency-Hospital.csv')
measures = ['OP-13', 'OP-8', 'OP-10']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-13': 'OP_13',
                       'OP-8': 'OP_8',
                       'OP-10': 'OP_10',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
oie_df1 = curate(oie_df)


4 remaining features: ['HCP_COVID_19_DEN', 'OP_10_DEN', 'OP_13_DEN', 'OP_8_DEN'] 



## MERGE DATAFRAME AND COMPARE TO SAS FILE

In [13]:
print(dates_df.shape)
dates_df.to_csv(stars_dir + "2024/measure_dates/08_2025_measure_dates.csv", index=False)

(60, 3)


In [14]:
main_df = tec_df.merge(tec2_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(cad_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(HCAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(uhv_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(hai_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(oie_df1, on='PROVIDER_ID', how='outer')

main_df['OP_10_DEN'] = np.nan
main_df['OP_13_DEN'] = np.nan
main_df['OP_8_DEN'] = np.nan
main_df['HCP_COVID_19_DEN'] = np.nan

for c in list(main_df):
    try:
        sas_cols_2026.remove(c)
    except:
        pass

print(len(sas_cols_2026), 'remaining features:', sorted(sas_cols_2026), '\n')
print(main_df.shape)
main_df.head()

0 remaining features: [] 

(4756, 91)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,HCP_COVID_19,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,HCP_COVID_19_DEN,ePC_02,ePC_02_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10,OP_10_DEN,OP_13_DEN,OP_8_DEN
0,010001,96,4115,215,373,5,57084,Not Available,Not Available,72,29,67,147,0.3,12,4303,NaN,Not Available,Not Available,11.4,270,3,144,9.4,112,10.2,583,18.4,517,13.5,395,194.78,125,3.2,27,0.98,Not Applicable,3,3,2,3,3,4,4.0,3.0,-15.6,273,-1.1,652,17.4,507,12.8,218,8.5,261,5.7,261,0.8,647,10.1,137,18,122,4.8,25,15.1,2824,9.096,8593,0.550,22.749,15644,0.176,6.089,213,0.493,0.922,98,Not Available,10.671,106573,0.187,66.908,106573,0.389,3.8,30.8,5.3,NaN,NaN,NaN
1,010005,72,2407,139,1125,3,58624,43,14,100,210,74,278,0.2,16,2244,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,8.9,126,14.1,158,21.2,285,12.9,89,191.14,43,3,104,1.01,Not Applicable,3,4,2,2,4,3,3.0,3.5,Not Available,Not Available,12.2,164,-17.2,292,14.2,897,8.6,99,6.2,99,1,396,Not Available,Not Available,17.1,132,4.2,81,13.3,986,2.599,4278,0.385,2.925,6429,0.684,2.583,96,0.000,0.541,52,Not Available,1.136,35407,2.641,8.903,33826,0.449,3.3,42.2,12.8,NaN,NaN,NaN
2,010006,66,2560,156,342,1,44924,62,16,86,85,62,140,0.5,16,3521,NaN,Not Available,Not Available,14.5,266,5.4,79,8.7,160,12.5,413,19.6,659,12.4,258,201.13,102,4.7,49,1.16,Not Applicable,3,2,2,2,3,2,2.0,2.5,-18,285,-4.7,461,-0.9,679,11.6,1618,Not Available,Not Available,Not Available,Not Available,1,502,10.6,71,19.1,174,5.1,56,15.9,2494,4.625,4186,0.000,9.980,7721,0.000,2.036,77,0.000,1.969,204,0.000,5.401,68560,0.000,38.827,65509,0.077,4.1,Not Available,8.8,NaN,NaN,NaN
3,010007,26,345,113,854,2,12667,Not Available,Not Available,65,52,11,18,0,Not Available,Not Available,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,11.2,34,13.4,34,25.4,98,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,0.93,Not Applicable,4,5,4,4,4,4,3.0,3.0,Not Available,Not Available,77.9,39,29.8,84,12.5,117,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,18.6,32,Not Available,Not Available,15.4,189,0.118,192,Not Available,0.808,1480,Not Available,0.139,5,Not Available,Not Available,Not Available,Not Available,0.091,4652,Not Available,1.759,4652,0.569,Not Available,Not Available,5.9,NaN,NaN,NaN
4,010008,39,126,114,343,1,6062,Not Available,Not Available,67,12,Not Available,Not Available,0.8,Not Available,Not Available,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,15,31,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Applicable,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,NaN,NaN,Not Available,Not Available,Not Available,Not Available,-8.9,29,12.8,60,Not Available,Not Available,Not Available,Not Available,Not Available,Not Availab

In [15]:

cols = []
for col in list(main_df):
    if 'PROVIDER_ID' in col:
        continue
    elif '_DEN' in col:
        continue
    else:
        cols.append(col)
        
print(sorted(cols), '\n')
print(len(cols), 'measures')

['COMP_HIP_KNEE', 'EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'Hybrid_HWR', 'IMM_3', 'MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 'MORT_30_PN', 'MORT_30_STK', 'OP_10', 'OP_13', 'OP_18B', 'OP_22', 'OP_23', 'OP_29', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 'OP_8', 'PSI_4_SURG_COMP', 'PSI_90_SAFETY', 'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'SAFE_USE_OF_OPIOIDS', 'SEP_1', 'ePC_02'] 

46 measures


In [16]:
print(main_df.shape)
main_df.dropna(how='all', subset = cols, inplace=True)
print(main_df.shape)

labs = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
        'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
        'OP_23', 'OP_29', 'IMM_3', 'SEP_1', 'MORT_30_CABG',
        'READM_30_CABG', 'OP_8', #'READM_30_HOSP_WIDE',
        'OP_10', 'OP_13', 
        'HCP_COVID_19', 
        'SAFE_USE_OF_OPIOIDS',
        'ePC_02', 
        'Hybrid_HWR',
        #'ePC_07b', 'ePC_07a', 'PC_05', 
        #'Hybrid_HWM',
        #'HH-01', 'HH-02',
       ]

summary = {}
for col in labs:
    series = main_df[col]
    
    # Count NaNs and "Not Available"
    n_nans = series.isna().sum()
    n_notavail = (series == 'Not Available').sum()
    
    # Coerce to numeric (turns "Not Available" into NaN)
    numeric = pd.to_numeric(series, errors='coerce')
    
    # Count valid numeric entries
    n_numeric = numeric.notna().sum()
    
    summary[col] = {
        'min': numeric.min(skipna=True),
        'max': numeric.max(skipna=True),
        'mean': numeric.mean(skipna=True),
        'median': numeric.median(skipna=True),
        'numeric_count': n_numeric,
        'NaN_count': n_nans,
        "'Not Available'_count": n_notavail
    }

summary_df = pd.DataFrame(summary).T
print(summary_df)


(4756, 91)
(4756, 91)
                      min    max       mean  median  numeric_count  NaN_count  \
READM_30_HIP_KNEE     2.4    7.8   4.855474     4.8         1626.0        1.0   
READM_30_COPD        15.7   23.9  18.227663    18.1         2704.0        1.0   
MORT_30_STK           8.1   22.1  13.235634    13.1         2217.0        0.0   
MORT_30_PN            7.6   32.2  16.352281    16.2         3661.0        0.0   
MORT_30_HF            5.0   20.2  11.575795    11.5         3115.0        0.0   
MORT_30_COPD          4.5   16.5   8.877561     8.7         2665.0        0.0   
MORT_30_AMI           6.7   17.1  12.139653    12.1         1957.0        0.0   
COMP_HIP_KNEE         1.4    9.3   3.614725     3.5         1657.0        0.0   
OP_22                 0.0  100.0   1.968703     1.0         3962.0       24.0   
OP_23                 0.0  100.0  70.955629    75.0         1510.0      128.0   
OP_29                 0.0  100.0  91.839523    97.0         2935.0       33.0   
IMM_3 

In [17]:
print(main_df.shape)
main_df.dropna(how='all', subset = cols, inplace=True)
print(main_df.shape)

ls = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
      'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
      'OP_23', 'OP_29', 'IMM_3', 'SEP_1', 'MORT_30_CABG', #'PC_01',
      'READM_30_CABG', 'Hybrid_HWR', 'OP_8',
      'OP_10', 'OP_13', 'HCP_COVID_19', 'SAFE_USE_OF_OPIOIDS',
      
      'ePC_02',
      'Hybrid_HWR',
     ]

for col in list(main_df):
    if col != 'PROVIDER_ID':
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce').astype(float)
        
for l in ls: 
    main_df[l] = main_df[l] * 0.01


(4756, 91)
(4756, 91)


# Notes
46 measures in 2025

### Lost 2 few measures:

READM_30_HOSP_WIDE: Rate of readmission after discharge from hospital (hospital-wide)

PC_01: Percentage of mothers whose deliveries were scheduled too early (1-2 weeks early), when a scheduled
delivery wasn’t medically necessary

### Including 2 that may show up in 2026:

Hybrid_HWR: Hybrid Hospital-Wide All-Cause Readmission Rate

ePC_02: The rate of nulliparous women with a term, singleton baby in a vertex position delivered by C-section
birth

### Optional but not included here:

Hybrid_HWM: Hybrid Hospital-Wide All-Cause Risk Standardized Mortality Rate

HH-01: Proportion of patients who are at risk and who had a low blood glucose test result (less than 40 mg/dL)
and no subsequent confirmatory blood glucose within 5 minutes and in the normal range (greater than 80
mg/dL) -- NO DATA

HH-02: Number of hospital days with a severe hyperglycemic event among the total qualifying hospital days for
at risk inpatient encounters -- NO DATA

ePC_07a: Rate of any serious complications for mothers during delivery (per 10,000 deliveries)

ePC_07b: Rate of serious complications (excluding blood transfusions) for mothers during delivery (per 10,000
deliveries) -- REDUNDANT WITH ePC_07a


In [18]:
prvdrs = []
for p in main_df['PROVIDER_ID'].tolist():
    if 'F' in p:
        p = p[:-1]
        p = p + '666666'
    prvdrs.append(p)

main_df['PROVIDER_ID'] = prvdrs

main_df['PROVIDER_ID'] = pd.to_numeric(main_df['PROVIDER_ID'], errors='coerce')
main_df.sort_values(by=['PROVIDER_ID'], ascending = True, inplace = True)
main_df.to_csv(stars_dir + "Reproduce_Stars_Input/2026/Input_File/data_for_2026_prognostications_from_Aug2025.csv", 
               index=False)


In [19]:
main_df.head()

,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,HCP_COVID_19,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,HCP_COVID_19_DEN,ePC_02,ePC_02_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10,OP_10_DEN,OP_13_DEN,OP_8_DEN
0,10001,0.96,4115.0,215.0,373.0,0.05,57084.0,NaN,NaN,0.72,29.0,0.67,147.0,0.003,0.12,4303.0,NaN,NaN,NaN,0.114,270.0,0.030,144.0,0.094,112.0,0.102,583.0,0.184,517.0,0.135,395.0,194.78,125.0,0.032,27.0,0.98,NaN,3.0,3.0,2.0,3.0,3.0,4.0,4.0,3.0,-15.6,273.0,-1.1,652.0,17.4,507.0,12.8,218.0,8.5,261.0,5.7,261.0,0.8,647.0,0.101,137.0,0.180,122.0,0.048,25.0,0.00151,2824.0,9.096,8593.0,0.550,22.749,15644.0,0.176,6.089,213.0,0.493,0.922,98.0,NaN,10.671,106573.0,0.187,66.908,106573.0,0.389,0.038,0.308,0.053,NaN,NaN,NaN
1,10005,0.72,2407.0,139.0,1125.0,0.03,58624.0,0.43,14.0,1.00,210.0,0.74,278.0,0.002,0.16,2244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.089,126.0,0.141,158.0,0.212,285.0,0.129,89.0,191.14,43.0,0.030,104.0,1.01,NaN,3.0,4.0,2.0,2.0,4.0,3.0,3.0,3.5,NaN,NaN,12.2,164.0,-17.2,292.0,14.2,897.0,8.6,99.0,6.2,99.0,1.0,396.0,NaN,NaN,0.171,132.0,0.042,81.0,0.00133,986.0,2.599,4278.0,0.385,2.925,6429.0,0.684,2.583,96.0,0.000,0.541,52.0,NaN,1.136,35407.0,2.641,8.903,33826.0,0.449,0.033,0.422,0.128,NaN,NaN,NaN
2,10006,0.66,2560.0,156.0,342.0,0.01,44924.0,0.62,16.0,0.86,85.0,0.62,140.0,0.005,0.16,3521.0,NaN,NaN,NaN,0.145,266.0,0.054,79.0,0.087,160.0,0.125,413.0,0.196,659.0,0.124,258.0,201.13,102.0,0.047,49.0,1.16,NaN,3.0,2.0,2.0,2.0,3.0,2.0,2.0,2.5,-18.0,285.0,-4.7,461.0,-0.9,679.0,11.6,1618.0,NaN,NaN,NaN,NaN,1.0,502.0,0.106,71.0,0.191,174.0,0.051,56.0,0.00159,2494.0,4.625,4186.0,0.000,9.980,7721.0,0.000,2.036,77.0,0.000,1.969,204.0,0.0,5.401,68560.0,0.000,38.827,65509.0,0.077,0.041,NaN,0.088,NaN,NaN,NaN
3,10007,0.26,345.0,113.0,854.0,0.02,12667.0,NaN,NaN,0.65,52.0,0.11,18.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.112,34.0,0.134,34.0,0.254,98.0,NaN,NaN,NaN,NaN,NaN,NaN,0.93,NaN,4.0,5.0,4.0,4.0,4.0,4.0,3.0,3.0,NaN,NaN,77.9,39.0,29.8,84.0,12.5,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.186,32.0,NaN,NaN,0.00154,189.0,0.118,192.0,NaN,0.808,1480.0,NaN,0.139,5.0,NaN,NaN,NaN,NaN,0.091,4652.0,NaN,1.759,4652.0,0.569,NaN,NaN,0.059,NaN,NaN,NaN
4,10008,0.39,126.0,114.0,343.0,0.01,6062.0,NaN,NaN,0.67,12.0,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.150,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.9,29.0,12.8,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00147,88.0,0.026,44.0,NaN,0.203,374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.062,2076.0,NaN,0.381,2076.0,NaN,NaN,NaN,0.019,NaN,NaN,NaN


In [20]:
ls = sorted(main_df['ePC_02'].unique().tolist())

print(len(ls))
print(ls)


33
[nan, 0.0, 0.05, 0.06, 0.07, 0.08, 0.11, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.35000000000000003, 0.36, 0.38, 0.45, 0.73]
